In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from numerate import *
from numerate.limiters import *
from numerate.functions import *
from numerate.verification import *

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 30.0

In [ ]:
g = lambda x : tophat(x, b=0.2, c=0.7)
h = lambda x : gaussian(x, b=0.05, c=0.3)
f = lambda x : g(x) + h(x)

In [ ]:
a = 1
x0 = 0
x1 = 1
xs = 300
revolutions = 1
ts = 500 * revolutions
func = f

In [ ]:
def div(a, b, epsilon):
    c = np.zeros_like(a)

    cond = np.logical_and(np.abs(a) <= epsilon, np.abs(b) <= epsilon)
    c[cond] = 1

    cond = np.logical_and(np.abs(a) > epsilon, np.abs(b) <= epsilon)
    c[cond] = np.sign(b)[cond] * a[cond] / epsilon

    cond = np.abs(b) > epsilon
    c[cond] = a[cond] / b[cond]

    return c

In [ ]:
def solve(solver, *, drop=3, interval=30, points=25, filename=None, phi=lax_wendroff, **kwargs):
    eq = solver(
        a,
        func,
        x0=x0,
        x1=x1,
        xs=xs,
        revolutions=revolutions,
        ts=ts,
        phi=phi,
        **kwargs,
    )
    solution = eq.solve()
    
    deltas = solution.toarray() - np.roll(solution.toarray(), 1)
    thetas = div(np.roll(deltas, 1), deltas, eq.epsilon)
    phis = phi(thetas, **kwargs)
        
    fig, ax = plt.subplots()

    sol_line, = ax.plot(
        eq.x_range, 
        solution[:, 0].toarray(), 
    )
    phi_scat = ax.scatter(
        eq.x_range, 
        phis[:, 0],
        s=0.75,
        c='red',
    )
    n_label = ax.text(0, 1, "$n = 0$", transform=ax.transAxes, fontsize=13)
    
#     plt.ylim(
#         np.min(solution.toarray().flatten()) - plt.rcParams['axes.ymargin'],
#         np.max(solution.toarray().flatten()) + plt.rcParams['axes.ymargin'],
#     )
    plt.ylim(
        0 - plt.rcParams['axes.ymargin'],
        2 + plt.rcParams['axes.ymargin'],
    )
    
    def anim_func(i):
        sol_line.set_ydata(solution[:, i].toarray())
        phi_scat.set_offsets(np.column_stack((eq.x_range, phis[:, i])))
        n_label.set_text(f"$n = {i}$")
                
        return sol_line, phi_scat, n_label,
    
    anim = FuncAnimation(
        fig,
        anim_func,
        frames=range(0, eq.ts, drop),
        interval=drop * interval / eq.a,
        blit=True,
        repeat=False
    )
        
    plt.close()
    
    if filename:
        writergif = PillowWriter(fps=15)
        anim.save(filename, writer=writergif)

    return anim

In [ ]:
solve(
    NumericalAdvectionEquationFluxLimiter,
    points=300,
#     filename="figures/presentation/thetas.gif",
    phi=beam_warming,
#     beta=2.0,
    epsilon=1e-12,
)